In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [61]:
# Joe Flacco, Andrew Luck, Cam Newton, Blake Bortles, Alex Smith, Blaine Gabbert. 
# Extract career stats information. Goal is to add contract years to contract df. Right, all we are doing is extracting contract
# year and salary. 

# Function to scrape QB data 
def qb_stats(name, url = None):
    if url is not None:
        url = url
    else:
        url = 'https://www.pro-football-reference.com/players/{}/{}00.htm'.format(name.split()[1][0], name.split()[1][:4] + name.split()[0][:2])
    request = requests.get(url).text
    soup = bs(request, features='lxml')

    headers = [th.text for th in soup.findAll('tr')[0].findAll('th')]
    index = [th.text for th in soup.find('table').findAll('th')[32:len(soup.find('table').findAll('th'))]]
    index = [x.strip('+').strip('*') for x in index]
    stats = [[td.text for td in tr.findAll('td')] for tr in soup.find('table').findAll('tr')[1:len(soup.find('table').findAll('tr'))]]
        
    dataframe = pd.DataFrame(stats, index = pd.Index(index, name = 'Year'),  columns = headers[1:len(headers)])
    return dataframe
def contract_dataframe(name, contract_year, salary, qb_df):
    # Accepts contract_year, salary, and qb_df to create dataframe of contract year stats.
    # Contract year is a string. 
    contract_year_df = qb_df.loc[[contract_year]]
    contract_year_df.insert(loc = 1, column = 'Player', value = name)
    contract_year_df.insert(loc = 32, column = 'Salary', value = salary )
    return contract_year_df

In [62]:
contract_dataframe('Joe Flacco', '2015',2220000, qb_stats('Joe Flacco'))

,Age,Player,Tm,Pos,No.,G,GS,QBrec,Cmp,Att,...,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD,AV,Salary
Year,,,,,,,,,,,,,,,,,,,,,
2015,30,Joe Flacco,BAL,QB,5,10,10,3-7-0,266,413,...,49.7,16,124,6.22,5.61,3.7,3,3,5,2220000


In [75]:
qb_dict = {'Joe Flacco':['2015', 22133333], 'Andrew Luck':['2015',24594000], 'Cam Newton':['2014', 20760000], 
          'Blake Bortles':['2018', 1000000], 'Alex Smith':['2017', 23500000], 'Blaine Gabbert':['2018',1000000]}

In [77]:
d = pd.DataFrame()
for qb, values in qb_dict.items():
    if qb == 'Alex Smith':
        a = contract_dataframe(qb, qb_dict[qb][0], qb_dict[qb][1], qb_stats(qb, 'https://www.pro-football-reference.com/players/S/SmitAl03.htm'))
    else:
        a = contract_dataframe(qb, qb_dict[qb][0], qb_dict[qb][1], qb_stats(qb))
    d = pd.concat([d, a])
    

In [68]:
qb_dict['Joe Flacco']

['2015', 22133333]

In [79]:
d.to_csv(r'C:\Users\Nate P\Desktop\NFL Salary Project FINAL\qb_contracts_quick_fix.csv')